In [2]:
from elasticsearch import Elasticsearch

In [4]:
es = Elasticsearch('http://localhost:9200')

In [ ]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch('http://localhost:9200')

In [5]:
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'a10681655ac6',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'AU4jtfjvRgmvfKuiL4bcUg',
 'version': {'number': '7.14.1',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '66b55ebfa59c92c15db3f69a335d500018b3331e',
  'build_date': '2021-08-26T09:01:05.390870785Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
import pprint  
INDEX_NAME = "toy_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "decompound_mode": "mixed",
            "filter": [ "pos_filter_speech", "nori_readingform",
            "lowercase", "remove_duplicates"],

          }
        },
        "filter": {
            "pos_filter_speech": {
                "type": "nori_part_of_speech",
           "stoptags":[
                        "E",
                        "IC",
                        "J",
                        "MAG",
                        "MM",
                        "NA",
                        "NR",
                        "SC",
                        "SE",
                        "SF",
                        "SH",
                        "SL",
                        "SN",
                        "SP",
                        "SSC",
                        "SSO",
                        "SY",
                        "UNA",
                        "UNKNOWN",
                        "VA",
                        "VCN",
                        "VCP",
                        "VSV",
                        "VV",
                        "VX",
                        "XPN",
                        "XR",
                        "XSA",
                        "XSN",
                        "XSV"
                    ]
          }
        }
      }
    }
  }
  ,
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }

  }
}

In [8]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'toy_index'}

In [25]:
chat_file = '../data/answer_total.txt'
chat_dict = {}
with open(chat_file, "r") as f:
    lines = f.readlines()
    for idx,line in enumerate(lines):
        text_dict = {}
        text_dict['text'] = line
        chat_dict[idx]= text_dict

In [26]:
chat_dict

{0: {'text': '회사 일에 집중할 수 없어 괴로우시겠어요.\n'},
 1: {'text': '아버지가 하신 말로인해 상처를 받으셨군요. 어떤 말인지 알려줄 수 있나요?\n'},
 2: {'text': '많이 힘드신 일이 있나 봐요. 회사에서 무슨 일이 있었나요?\n'},
 3: {'text': '열심히 공부하셔서 다음에는 좋은 결과가 있길 바라요.\n'},
 4: {'text': '그 사람만이 알고 있을 사실이네요.\n'},
 5: {'text': '얼마 전에 면접 본 곳에서 합격했다는 소식을 듣게 되어 기쁘고 신나셨군요.\n'},
 6: {'text': '상황이 많이 안 좋군요.\n'},
 7: {'text': '돈을 빌려준 친구에게 갚으라는 말을 들었군요.\n'},
 8: {'text': '믿는 친구에게 소개를 받아서 기쁘시겠어요.\n'},
 9: {'text': '잘 치료받아 원하는 대로 하실 수 있길 바랄게요.\n'},
 10: {'text': '계속 함께 있던 아내에게 배신감을 느끼시는 것 같군요.\n'},
 11: {'text': '선생님이 오히려 전학을 권하셨군요. 많이 화가 나셨겠어요.\n'},
 12: {'text': '점보다 앞에 있는 사람을 믿어보세요.\n'},
 13: {'text': '여자친구의 바람으로 이별을 선택하셨군요.\n'},
 14: {'text': '친구들이 나한테 답을 물어볼 정도로 국어는 자신 있군요. 지금 상황에서 시도해볼 만한 것이 뭐가 있을까요?\n'},
 15: {'text': '금리가 좋은 은행으로 옮기기를 고르셨군요.\n'},
 16: {'text': '회사의 인재라고 생각하시는군요. 좀 더 자세히 말씀해 주시겠어요?\n'},
 17: {'text': '헤어진 남자친구와 같은 수업을 듣게 되다니 정말 당황했겠어요.\n'},
 18: {'text': '좋은 실력을 가지고 계셔서 다음 대회도 문제없을 거예요.\n'},
 19: {'text': '당뇨 수치 때문에 걱정이 많으신 것 같아요. 이 문제에 대해 어

In [27]:
from tqdm import tqdm


for doc_id, doc in tqdm(chat_dict.items()):
    es.index(index=INDEX_NAME,  id=doc_id, body=doc)

  0%|          | 0/116179 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
100%|██████████| 116179/116179 [05:56<00:00, 325.73it/s]


In [28]:
doc = es.get(index=INDEX_NAME, id=1)
pprint.pprint(doc)

{'_id': '1',
 '_index': 'toy_index',
 '_primary_term': 1,
 '_seq_no': 1,
 '_source': {'text': '아버지가 하신 말로인해 상처를 받으셨군요. 어떤 말인지 알려줄 수 있나요?\n'},
 '_type': '_doc',
 '_version': 1,
 'found': True}


In [31]:
results = es.search(index=INDEX_NAME, body={'from':0, 'size':10, 'query':{'match':{'text':'정말 감사한일이죠'}}})
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source:', result['_source'])

score: 5.166352 source: {'text': '정말 좋은 회사군요. 정말 좋으시겠어요.\n'}
score: 5.166352 source: {'text': '정말 대견한 강아지에요. 정말 부러운걸요?\n'}
score: 5.166352 source: {'text': '정말 멋져요. 다이어트 정말 힘들잖아요.\n'}
score: 5.166352 source: {'text': '정말 좋은 기회였네요. 정말 기쁘시겠어요.\n'}
score: 4.946699 source: {'text': '정말 뿌듯하시겠어요. 그동안 정말 고생 많으셨어요.\n'}
score: 4.887418 source: {'text': '정말 잘됐네요!\n'}
score: 4.887418 source: {'text': '정말 충격적이네요.\n'}
score: 4.887418 source: {'text': '정말 뿌듯하시겠어요.\n'}
score: 4.887418 source: {'text': '정말 스트레스받으시겠어요.\n'}
score: 4.887418 source: {'text': '정말 좋아했나봐요.\n'}


In [30]:
query = "정말 감사한일이죠"
res = es.search(index=INDEX_NAME, q=query, size=3)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '32501',
                    '_index': 'toy_index',
                    '_score': 5.166352,
                    '_source': {'text': '정말 좋은 회사군요. 정말 좋으시겠어요.\n'},
                    '_type': '_doc'},
                   {'_id': '34638',
                    '_index': 'toy_index',
                    '_score': 5.166352,
                    '_source': {'text': '정말 대견한 강아지에요. 정말 부러운걸요?\n'},
                    '_type': '_doc'},
                   {'_id': '102780',
                    '_index': 'toy_index',
                    '_score': 5.166352,
                    '_source': {'text': '정말 멋져요. 다이어트 정말 힘들잖아요.\n'},
                    '_type': '_doc'}],
          'max_score': 5.166352,
          'total': {'relation': 'eq', 'value': 3610}},
 'timed_out': False,
 'took': 2}
